Übersetzen
To-Dos (Preprocessing):
0. removed duplicate songs
1. Contractions (I'm -> I am) 
2. Tokenisierung (NLKT, Gensim, spaCy)
3. Spelling Correction
4. POS-Tagging
5. Stemming / Lemmatisierung (mit Spacy)
6. Lowercasing
7. Balance Data - Sprich dass alle Labels gleich viel Daten haben
8. word order??


Word2Vec and Word2Vec with TFID

To-Dos (EDA):
- Schaue Folie
1. analyzed word counts (Only 27 of the rock words and 10 of the pop words out of the 300 most frequently appearing words
for those genres were unique to those genres, suggesting that a bag-of-words embedding or a machine
learning model that didn’t utilize the order of the words would not be very effective.)

- Plots (This plots the words and their respective counts in the pop genre that are not part of the
top 300 words for hip hop or rock.)

To-Dos (Feature Engineering):
-  One-Hot Encoding (Sätze)
- Bag-of-Words (BoW) (ist eine Methode zur Darstellung von Textdaten in numerischer Form)
- N-Grams
- TF-IDF (statistische Methode zur Bewertung der Wichtigkeit eines Wortes)
- Cosine Similarity – Wortähnlichkeiten berechnen
- Embeeding mit (Word2Vec,GloVe,fastText )


## Imports

In [49]:
%pip install -r requirements.txt
# Load, explore and plot data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
import re

######## SEED setzen
import random
# Random Seed setzen:
random.seed(49)
np.random.seed(49)
SEED = 49

Note: you may need to restart the kernel to use updated packages.


## Load Data

### Dataset 1

In [50]:
df_single_label = pd.read_csv('preprocessed_single_label.csv')
df_single_label.head(2)

,Unnamed: 0,Titel,Lyric,Artist,Genres
0,1719,I Miss Her,oh lord id like know thinks oh oh lord wanna g...,Olodum,Axé
1,1766,They Don't Care About Us,skin head dead head everybody gone bad situati...,Olodum,Axé


In [51]:
df_single_label.drop("Unnamed: 0", axis=1, inplace=True)
df_single_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42760 entries, 0 to 42759
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Titel   42758 non-null  object
 1   Lyric   42758 non-null  object
 2   Artist  42760 non-null  object
 3   Genres  42760 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [52]:
df_single_label.isna().sum()

Titel     2
Lyric     2
Artist    0
Genres    0
dtype: int64

In [53]:
df_single_label = df_single_label.dropna(subset=['Lyric'])
df_single_label.isna().sum()

Titel     2
Lyric     0
Artist    0
Genres    0
dtype: int64

### Dataset 2

In [54]:
df_split_label = pd.read_csv('preprocessed_split_label.csv')
df_split_label.head(2)

,Unnamed: 0,Titel,Lyric,Artist,Genres
0,69,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Pop
1,69,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Axé


In [55]:
df_split_label.drop("Unnamed: 0", axis=1, inplace=True)
df_split_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481895 entries, 0 to 481894
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Titel   481893 non-null  object
 1   Lyric   481890 non-null  object
 2   Artist  481895 non-null  object
 3   Genres  481895 non-null  object
dtypes: object(4)
memory usage: 14.7+ MB


In [56]:
df_split_label.isna().sum()

Titel     2
Lyric     5
Artist    0
Genres    0
dtype: int64

In [57]:
df_split_label = df_split_label.dropna(subset=['Lyric'])
df_split_label.isna().sum()

Titel     2
Lyric     0
Artist    0
Genres    0
dtype: int64

## EDA

### Dataset 1

Lyric Count Variable:

In [58]:
df_single_label['Lyric_count'] = df_single_label['Lyric'].map(lambda x: len(x.split()))
df_single_label.head(2)

,Titel,Lyric,Artist,Genres,Lyric_count
0,I Miss Her,oh lord id like know thinks oh oh lord wanna g...,Olodum,Axé,63
1,They Don't Care About Us,skin head dead head everybody gone bad situati...,Olodum,Axé,265


Normalisieren der Lyrics fürs plotting

In [59]:
df_single_label['Count_norm'] = df_single_label['Lyric_count'] / df_single_label['Lyric_count'].max()
df_single_label['Count_norm'] = df_single_label['Count_norm'].map(lambda x: x*55)
df_single_label.head(2)

,Titel,Lyric,Artist,Genres,Lyric_count,Count_norm
0,I Miss Her,oh lord id like know thinks oh oh lord wanna g...,Olodum,Axé,63,2.547794
1,They Don't Care About Us,skin head dead head everybody gone bad situati...,Olodum,Axé,265,10.716912


### Dataset 2

In [60]:
df_split_label['Lyric_count'] = df_split_label['Lyric'].map(lambda x: len(x.split()))
df_split_label.head(2)

,Titel,Lyric,Artist,Genres,Lyric_count
0,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Pop,164
1,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Axé,164


In [61]:
df_split_label['Count_norm'] = df_split_label['Lyric_count'] / df_split_label['Lyric_count'].max()
df_split_label['Count_norm'] = df_split_label['Count_norm'].map(lambda x: x*55)
df_split_label.head(2)

,Titel,Lyric,Artist,Genres,Lyric_count,Count_norm
0,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Pop,164,3.653301
1,Careless Whisper,feel unsure take hand lead dance floor music d...,Ivete Sangalo,Axé,164,3.653301
